In [2]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *


from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

In [3]:
desde = '2022-04-28'
hasta = '2022-07-28'
city = 'BOG'

In [4]:
query = """
WITH 
count_fulfillments_pasados AS (
	SELECT 
	    COUNT(DISTINCT boi.order_item_id) AS count_items,
	    bfg.order_id
     
	FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"              bfg
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order"                     bo      ON bo.order_id = bfg.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"          ffg     ON bfg.fulfillment_group_id = ffg.fulfillment_group_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"         bfo     ON bfg.fulfillment_group_id = bfo.fulfillment_group_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"             bop     ON bop.order_id = bo.order_id
	LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order"                      fo      ON fo.order_id = bo.order_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON bo.site_disc = s.site_id
	LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                fot     ON fot.fb_order_type_id=fo.fb_order_type_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"   bfgi    ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
	INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON bfgi.order_item_id= boi.order_item_id
	
	WHERE      
        ffg.close_date between '{desde}' AND '{hasta}'
        --AND s.site_identifier_value = {city}
        AND bo.order_status='SUBMITTED'
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
        AND fo.fb_order_status_id IN (1,6,7,8)
        AND bop.archived = 'N'
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
	GROUP BY 2
),

ful AS (

    SELECT DISTINCT
        bo.order_id,
        ffg.close_date
        
    FROM postgres_broadleaf_federate."broadleaf.blc_order"                      bo
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg     ON bfg.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo     ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo      ON fo.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop     ON bop.order_id = bo.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s       ON s.site_id = bo.site_disc
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot     ON fot.fb_order_type_id=fo.fb_order_type_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment"     ba      ON ba.order_id = bo.order_id
    
    WHERE fo.fb_order_status_id IN (1,6,7,8)
        AND ffg.close_date between '{desde}' AND '{hasta}'
        --AND s.site_identifier_value = {city}
        AND bo.order_status = 'SUBMITTED'
        AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
        AND bop.archived = 'N'
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
        
    GROUP BY 1,2
)

SELECT DISTINCT
    --bo.order_id,
    bo.customer_id,
    s.site_identifier_value as region_code,
    --TO_CHAR(ful.close_date, 'YYYY-mm-dd') AS day,
    bcat2.name as cat,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    bs.sku_id,
    bs.addl_product_id AS padre_sku_id,
    bs.name AS product,
    SUM(boi.quantity) AS quantity,
    CASE 
        WHEN s.site_identifier_value = 'CMX' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'GDL' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'PBC' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'MTY' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
        WHEN s.site_identifier_value = 'SPO' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'BHZ' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'CWB' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        WHEN s.site_identifier_value = 'VCP' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
    ELSE ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3000) END AS net_gmv_usd
    
FROM 
    postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"                  bfgi
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"            bfg     ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"             ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                        bo      ON bo.order_id = bfg.order_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON bo.site_disc = s.site_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"                   boi     ON bfgi.order_item_id= boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                    foi     ON boi.order_item_id= foi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                         fow     ON fow.owner_id = ffg.owner_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                         fo      ON fo.order_id = bo.order_id
    INNER JOIN ful                                                                              ON ful.order_id = bo.order_id
    LEFT JOIN count_fulfillments_pasados                                                cf      ON bfg.order_id = cf.order_id
    LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                    fot     ON fot.fb_order_type_id=fo.fb_order_type_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"          bdoi    ON bdoi.order_item_id = boi.order_item_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bdoi.sku_id
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"                 bcx     ON bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and bcx.default_reference = 'true'
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"                      bcat    ON bp.default_category_id = bcat.category_id
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"                      bcat2   ON bcx.category_id = bcat2.category_id
    --LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"          boipd   ON boipd.order_item_id=boi.order_item_id
    --LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"            boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id
WHERE 
    ffg.close_date between '{desde}' AND '{hasta}'
    --AND s.site_identifier_value = {city}
    AND bo.order_status='SUBMITTED'
    AND fo.fb_order_status_id IN (1,6,7,8)
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND bcat2.name IN ('Abarrotes','Aseo e Higiene','Bebidas','Congelados','Desechables','Lácteos & Huevos','Carne, Pollo & Pescados', 'Frutas & Verduras', 'Bebidas','Mercearia','Limpeza e Higiene','Laticínios e Ovos','Frutas e Verduras','Congelados','Descartáveis','Carnes, Aves e Peixes')

GROUP BY 1,2,3,4,5,6,7--,8--,9
""".format(desde=desde, hasta=hasta, city=city)
df = read_connection_data_warehouse.runQuery(query)
df = df[(~df['region_code'].isin(['MTY']))]

In [5]:
#//----------------PRECARGA-------------------------//
#//Ruta de data // Base Data (Without Deleting) 
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df = df.merge(frida_products[['ciudad::multiFilter','status_frida','id_tarjeta', 'sku_id']], how = 'left', on = 'sku_id').dropna().copy()

200


In [6]:
#//----------------PRECARGA-------------------------//
#//Ruta de data // Base Data (Without Deleting) 
segmentos_full = get_fresh_query_result("https://internal-redash.federate.frubana.com/",101609,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [7]:
data_full = df.merge(segmentos_full[['customer_id','microsegment']], how = 'left', on = 'customer_id')
data_full['microsegment'] = data_full['microsegment'].fillna('Other')

In [8]:
#//Clean segment_names
cities_others = ['GDL', 'BHZ', 'PBC', 'MDE', 'VCP','CWB']

data_full.loc[data_full[data_full['region_code'].isin(cities_others)].index,('microsegment')] = 'Other'

data_full['microsegment'] = data_full['microsegment'].str.replace('s/café da manhã','')
data_full['microsegment'] = data_full['microsegment'].str.replace('c/café da manhã','')
data_full['microsegment'] = data_full['microsegment'].str.replace('c/desayuno','')
data_full['microsegment'] = data_full['microsegment'].str.replace('s/desayuno','')
data_full['microsegment'] = data_full['microsegment'].str.strip()
data_full['microsegment'] = data_full['microsegment'].apply(lambda x: 'Taquería' if 'Taquería' in x else x)
data_full['microsegment'] = data_full['microsegment'].apply(lambda x: x.replace('Parrilla/Asadero','Parrilla/asadero'))

data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')].apply(lambda x: x.replace('Empanadas/Arepas','Empanadas/arepas'))
data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')].apply(lambda x: x.replace('Fritos','Empanadas/arepas'))
data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')].apply(lambda x: 'Panadería/Pastelería' if 'Panadería/Pastelería/Cafetería' in x else x)


data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')].apply(lambda x: 'Panadería/Pastelería/Cafetería' if 'Cafetería' in x else x)
data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')].apply(lambda x: 'Panadería/Pastelería/Cafetería' if 'Panadería/Pastelería' in x else x)
data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BAQ'].index,('microsegment')].apply(lambda x: x.replace('Empanadas/Arepas','Fritos'))

data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')] = data_full.loc[data_full[data_full['region_code'] == 'BOG'].index,('microsegment')].apply(lambda x: x.replace('Empanadas/Arepas','Empanadas/arepas'))

In [9]:
data_full['subcat'] = data_full['subcat'].str.strip()
data_full.dropna(axis=0, subset=['customer_id'], inplace=True)
data_full['customer_id'] = data_full['customer_id'].astype('str')

In [10]:
def cross_sell(city, microsegment):
    """
    This function aim to find the relation between SKUs to increase the probability of buying both at the same time (cross_sell) 
    and its filtered by a segment of clients to aim a better understanding of the group

    Args:
        city (str): The city that you want to figure out its cross_sell of SKUs
        microsegment (str): The microsegment of the city

    Returns:
        SKUs: A list with the subcategories that generate the best mix for the cross_sell
        df: A data frame with the info of the  antecedents and consequents and their confidence values
    """
    
    basket = (data_full[(data_full['region_code'] == city) & (data_full['microsegment'] == microsegment)]
          .groupby(['customer_id', 'product'])['quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('customer_id'))
    
    def encode_units(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    basket_sets = basket.applymap(encode_units)
    
    frequent_itemsets = fpgrowth(basket_sets.astype(bool), min_support=0.1, use_colnames=True)
    frequent_itemsets
    
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.50)
     
    #support how many times of be together (P(AuC)) 
    #confidence is equal to P(C/A)
    #lift is to evaluate the support and confidence rull, there are times when both of these measures may be high, and yet still produce a rule that is not useful -> (Rule Support) /(Support(Antecedent) * Support(Consequent))
    """
    Any rule with an improvement of less than 1 does not indicate a real cross-selling opportunity,
    no matter how high its support and confidence, because it actually offers less ability to predict a purchase than does random chance.
    """
    antecedents = list(map(list, rules.antecedents.values))  
    antecedents1 = set([x for xs in antecedents for x in xs])

    consequents = list(map(list, rules.consequents.values))  
    consequents1 = set([x for xs in consequents for x in xs])

    set3 = antecedents1.union(consequents1)
    
    return list(set3), rules.sort_values(by = 'confidence', ascending=False)
    

In [ ]:
parametros = {}
d = {}
for i in data_full['region_code'].unique():
    parametros[i] = {}
    print(i)
    for j in data_full[['region_code', 'microsegment']][(data_full['region_code'] == i)].drop_duplicates().values[:,1]:
        parametros[i][j], d["df_xsell_{city}_{ms}".format(city=i,ms=j)] = cross_sell(i, j)

In [ ]:
#parametros

In [13]:
d.keys()

dict_keys(['df_xsell_BOG_Comida rápida', 'df_xsell_BOG_Cafetería', 'df_xsell_BOG_Other', 'df_xsell_BOG_Comida especializada', 'df_xsell_BOG_Corrientazo', 'df_xsell_BOG_Panadería/Pastelería', 'df_xsell_BOG_Parrilla/asadero', 'df_xsell_BOG_Empanadas/arepas', 'df_xsell_CMX_Fonda', 'df_xsell_CMX_Other', 'df_xsell_CMX_Comida rápida', 'df_xsell_CMX_Cafetería', 'df_xsell_CMX_Panadería/Pastelería', 'df_xsell_CMX_Juguería/Comida saludable', 'df_xsell_CMX_Taquería', 'df_xsell_CMX_Comida especializada', 'df_xsell_SPO_Other', 'df_xsell_SPO_Prato do dia', 'df_xsell_SPO_Comida brasileira y saudavel', 'df_xsell_SPO_Padaria/Cafeteria/Bolos', 'df_xsell_SPO_Comida por quilo', 'df_xsell_SPO_Lanches', 'df_xsell_SPO_Comida internacional/Grill', 'df_xsell_SPO_Sucos/Açaí', 'df_xsell_SPO_Pizza/Comida Italiana', 'df_xsell_BHZ_Other', 'df_xsell_BAQ_Other', 'df_xsell_BAQ_Corrientazo', 'df_xsell_BAQ_Fritos', 'df_xsell_BAQ_Comida especializada', 'df_xsell_BAQ_Comida de mar y Colombiana', 'df_xsell_BAQ_Panadería/Pa

In [15]:
d['df_xsell_BAQ_Corrientazo'].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
13,"(Huevos AA x 30 Unid, Papa Blanca x kilo)",(Aceite Naoli 3LT),0.145238,0.629524,0.119524,0.822951,1.307259,0.028093,2.092504
31,(Frijol Zaragoza Naoli de 500gr),(Aceite Naoli 3LT),0.149524,0.629524,0.122381,0.818471,1.300144,0.028252,2.040869
54,(Arroz Naoli 3Kg),(Aceite Naoli 3LT),0.152857,0.629524,0.121905,0.797508,1.266843,0.025678,1.829582
64,(Arroz Naoli 500gx25un),(Aceite Naoli 3LT),0.203810,0.629524,0.162381,0.796729,1.265606,0.034078,1.822573
30,(Lenteja Naoli de 500gr),(Aceite Naoli 3LT),0.149524,0.629524,0.119048,0.796178,1.264731,0.024919,1.817649


In [16]:
# cities = {}
# for i in parametros:
#     aux = []
#     for j in parametros[i]:
#         aux.append(sorted(parametros[i][j]))
#     cities[i] = pd.DataFrame(aux).T
#     cities[i].columns = list(parametros[i].keys())
#     cities[i].to_csv(f'{i}_MS_SUBCAT.csv', index = False)